In [1]:
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [146]:
import pandas as pd 
dfDeliveries = pd.read_csv(r"C:\Users\hp\Desktop\MAJOR\data\IPL_Ball_by_Ball_2008_2022.csv") 
dfMatches = pd.read_csv(r"C:\Users\hp\Desktop\MAJOR\data\archive\IPL_Matches_2008_2022.csv")
dfDeliveries = dfDeliveries.merge(dfMatches, on='ID')
dfDeliveries['BowlingTeam'] = np.where(dfDeliveries['BattingTeam']==dfDeliveries['Team1'],
                                       dfDeliveries['Team2'],
                                       dfDeliveries['Team1'])
def overClassifier(val):
    if val>=0 and val<6:
        return 'Power Play'
    elif val>=6 and val<15:
        return 'Middle Overs'
    else:
        return 'Death Overs'
dfDeliveries['PhaseOfPlay'] = dfDeliveries['overs'].map(overClassifier)
dfDeliveries[['overs','PhaseOfPlay']].sort_values('overs').drop_duplicates().reset_index(drop=True)
dfDeathOvers = dfDeliveries[dfDeliveries['PhaseOfPlay']=='Death Overs']
dfPowerplayOvers = dfDeliveries[dfDeliveries['PhaseOfPlay']=='Power Play Overs']
dfMiddleOvers = dfDeliveries[dfDeliveries['PhaseOfPlay']=='Middle Overs']

In [64]:

dfBatsmen = dfDeliveries.groupby('batter')\
            .agg({'ID':'nunique','batsman_run':'sum','isWicketDelivery':'sum'})\
            .reset_index()
dfBatsmen.columns = ['Batter','Innings','Runs','Dismissals']
dfBallsFaced = dfDeliveries[(dfDeliveries['extra_type']!='wides')].groupby('batter')['ID'].count().reset_index()
dfBallsFaced.columns = ['Batter','Balls Faced']
dfBatter4s = dfDeliveries[(dfDeliveries['batsman_run']==4)&(dfDeliveries['non_boundary']==0)]\
            .groupby('batter')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfBatter4s.columns = ['Batter','4s']
dfBatter6s = dfDeliveries[(dfDeliveries['batsman_run']==6)&(dfDeliveries['non_boundary']==0)]\
            .groupby('batter')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfBatter6s.columns = ['Batter','6s']
dfMatchWise = dfDeliveries.groupby(['batter','ID'])['batsman_run'].sum().reset_index()
dfHighScore = dfMatchWise.groupby(['batter'])['batsman_run'].max().reset_index()
dfHighScore.columns = ['Batter','High Score']
df50s = dfMatchWise[(dfMatchWise['batsman_run']>=50)&(dfMatchWise['batsman_run']<100)]\
        .groupby('batter')['batsman_run']\
        .count()\
        .reset_index()\
        .sort_values('batsman_run',ascending=False)
df50s.columns = ['Batter','50']
df100s = dfMatchWise[(dfMatchWise['batsman_run']>=100)]\
        .groupby('batter')['batsman_run']\
        .count()\
        .reset_index()\
        .sort_values('batsman_run',ascending=False)
df100s.columns = ['Batter','100']
dfBatsmen = dfBatsmen.merge(dfBallsFaced, on='Batter',how='outer').merge(dfBatter4s, on='Batter',how='outer')\
                     .merge(dfBatter6s, on='Batter',how='outer').merge(dfHighScore, on='Batter',how='outer')\
                     .merge(df50s, on='Batter',how='outer').merge(df100s, on='Batter',how='outer')
dfBatsmen['Strike Rate'] = (dfBatsmen['Runs']/dfBatsmen['Balls Faced'])*100
dfBatsmen['Batting Average'] = dfBatsmen['Runs']/dfBatsmen['Dismissals']
dfBatsmen.fillna(0, inplace=True)
dfBatsmen.sort_values(by='Runs', ascending=False).head()

,Batter,Innings,Runs,Dismissals,Balls Faced,4s,6s,High Score,50,100,Strike Rate,Batting Average
569,V Kohli,215,6634,193,5133,580.0,218.0,113,45.0,5.0,129.242159,34.373057
462,S Dhawan,205,6244,179,4944,701.0,136.0,106,47.0,2.0,126.294498,34.882682
130,DA Warner,162,5883,143,4185,577.0,216.0,126,55.0,4.0,140.573477,41.139860
430,RG Sharma,221,5881,203,4529,519.0,240.0,109,40.0,1.0,129.852064,28.970443
493,SK Raina,200,5536,168,4046,506.0,204.0,100,39.0,1.0,136.826495,32.952381


In [163]:

dfbowler = dfDeliveries.groupby('bowler')\
            .agg({'ID':'nunique','batsman_run':'sum','isWicketDelivery':'sum'})\
            .reset_index()
dfbowler.columns = ['bowler','Innings','Runs','Dismissals']

dfBallsBowled = dfDeliveries[(dfDeliveries['extra_type']!='wides')].groupby('bowler')['ID'].count().reset_index()
dfBallsBowled.columns = ['bowler','Balls Bowled']

dfbowler4s = dfDeliveries[(dfDeliveries['batsman_run']==4)&(dfDeliveries['non_boundary']==0)]\
            .groupby('bowler')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfbowler4s.columns = ['bowler','4s']
dfbowler0s = dfDeliveries[(dfDeliveries['batsman_run']==0)&(dfDeliveries['non_boundary']==0)]\
            .groupby('bowler')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfbowler0s.columns = ['bowler','0s']
dfbowler6s = dfDeliveries[(dfDeliveries['batsman_run']==6)&(dfDeliveries['non_boundary']==0)]\
            .groupby('bowler')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfbowler6s.columns = ['bowler','6s']
dfMatchWise = dfDeliveries.groupby(['bowler','ID'])['batsman_run'].sum().reset_index()
dfHighScore = dfMatchWise.groupby(['bowler'])['batsman_run'].max().reset_index()
dfHighScore.columns = ['bowler','Highest Conceeded']

dfbowler = dfbowler.merge(dfBallsBowled, on='bowler',how='outer').merge(dfbowler4s, on='bowler',how='outer')\
                     .merge(dfbowler6s, on='bowler',how='outer').merge(dfbowler0s, on='bowler',how='outer').merge(dfHighScore, on='bowler',how='outer')
dfbowler['Strike Rate'] = (dfbowler['Balls Bowled']/dfbowler['Dismissals'])
dfbowler['Bowling Average'] = dfbowler['Runs']/dfbowler['Dismissals']
dfbowler['economy'] = (dfbowler['Runs']/dfbowler['Balls Bowled'])*6
dfbowler.fillna(0, inplace=True)
df=dfbowler
threshold = 25
new_df = df[df['Innings'] > (df['Innings'].shift(1) + threshold)]
new_df.sort_values(by='Strike Rate', ascending=False).head()








,bowler,Innings,Runs,Dismissals,Balls Bowled,4s,6s,0s,Highest Conceeded,Strike Rate,Bowling Average,economy
111,DJ Hooda,30,489,10,349,36.0,22.0,110.0,37,34.900000,48.900000,8.406877
390,SK Raina,69,1096,30,908,71.0,44.0,321.0,39,30.266667,36.533333,7.242291
65,B Lee,38,1042,30,888,123.0,25.0,435.0,43,29.600000,34.733333,7.040541
238,M Kartik,55,1341,39,1154,97.0,41.0,433.0,50,29.589744,34.384615,6.972270
183,JP Duminy,49,802,23,678,53.0,33.0,265.0,33,29.478261,34.869565,7.097345


In [149]:

dfDeliveries.head(19)

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,...,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2,BowlingTeam,PhaseOfPlay
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,...,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,Power Play
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,...,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,Power Play
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,...,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,Power Play
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,...,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,Power Play
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,...,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,Power Play
5,1312200,1,0,6,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,...,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,Power Play
6,1312200,1,1,1,JC Buttler,Yash Dayal,YBK Jaiswal,NaN,0,0,...,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,Power Play
7,1312200,1,1,2,JC Buttler,Yash Dayal,YBK Jaiswal,NaN,0,0,...,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,Power Play
8,1312200,1,1,3,JC Buttler,Yash Dayal,YBK Jaiswal,NaN,4,0,...,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,Power Play
9,1312200,1,1,4,JC Buttler,Yash Dayal,YBK Jaiswal,NaN,0,0,...,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,Power Play


In [43]:
dfDeathBowling = dfDeathOvers.groupby('BowlingTeam').agg({'total_run':'sum','isWicketDelivery':'sum'}).reset_index()
dfDeathBowling.columns = ['Team','Runs Conceded','Wickets']
dfDOBallsBowled = dfDeliveries[~((dfDeliveries['extra_type']=='wides'))&(dfDeliveries['PhaseOfPlay']=='Death Overs')]\
                  .groupby('BowlingTeam')['ID']\
                  .count().reset_index()
dfDOBallsBowled.columns = ['Team','Balls Bowled']
dfDeathBowling = dfDeathBowling.merge(dfDOBallsBowled, on='Team')
dfDeathBowling['Overs'] = dfDeathBowling['Balls Bowled']/6
dfDeathBowling['Bowling Average'] = dfDeathBowling['Runs Conceded']/dfDeathBowling['Wickets']
dfDeathBowling['Bowling Strike Rate'] = dfDeathBowling['Balls Bowled']/dfDeathBowling['Wickets']
dfDeathBowling['Economy Rate'] = dfDeathBowling['Runs Conceded']/dfDeathBowling['Overs']
dfDeathBowling = dfDeathBowling.sort_values('Economy Rate',ascending=False).reset_index(drop=True)
dfDeathBowling

,Team,Runs Conceded,Wickets,Balls Bowled,Overs,Bowling Average,Bowling Strike Rate,Economy Rate
0,Lucknow Super Giants,776,35,418,69.666667,22.171429,11.942857,11.138756
1,Gujarat Lions,1354,51,748,124.666667,26.549020,14.666667,10.860963
2,Rising Pune Supergiants,616,34,357,59.500000,18.117647,10.500000,10.352941
3,Deccan Chargers,3451,166,2004,334.000000,20.789157,12.072289,10.332335
4,Kings XI Punjab,8135,409,4828,804.666667,19.889976,11.804401,10.109776
5,Royal Challengers Bangalore,9711,491,5813,968.833333,19.778004,11.839104,10.023396
6,Rajasthan Royals,8217,394,4939,823.166667,20.855330,12.535533,9.982183
7,Punjab Kings,1126,59,681,113.500000,19.084746,11.542373,9.920705
8,Delhi Capitals,2932,160,1800,300.000000,18.325000,11.250000,9.773333
9,Delhi Daredevils,6535,346,4069,678.166667,18.887283,11.760116,9.636274


In [44]:
dfMiddleBowling = dfMiddleOvers.groupby('BowlingTeam').agg({'total_run':'sum','isWicketDelivery':'sum'}).reset_index()
dfMiddleBowling.columns = ['Team','Runs Conceded','Wickets']
dfMOBallsBowled = dfDeliveries[~((dfDeliveries['extra_type']=='wides'))&(dfDeliveries['PhaseOfPlay']=='Middle Overs')]\
                  .groupby('BowlingTeam')['ID']\
                  .count().reset_index()
dfMOBallsBowled.columns = ['Team','Balls Bowled']
dfMiddleBowling = dfMiddleBowling.merge(dfMOBallsBowled, on='Team')
dfMiddleBowling['Overs'] = dfMiddleBowling['Balls Bowled']/6
dfMiddleBowling['Bowling Average'] = dfMiddleBowling['Runs Conceded']/dfMiddleBowling['Wickets']
dfMiddleBowling['Bowling Strike Rate'] = dfMiddleBowling['Balls Bowled']/dfMiddleBowling['Wickets']
dfMiddleBowling['Economy Rate'] = dfMiddleBowling['Runs Conceded']/dfMiddleBowling['Overs']
dfMiddleBowling = dfMiddleBowling.sort_values('Economy Rate',ascending=False).reset_index(drop=True)
dfMiddleBowling


,Team,Runs Conceded,Wickets,Balls Bowled,Overs,Bowling Average,Bowling Strike Rate,Economy Rate
0,Gujarat Lions,2235,51,1621,270.166667,43.823529,31.784314,8.272671
1,Gujarat Titans,1151,36,860,143.333333,31.972222,23.888889,8.030233
2,Punjab Kings,1978,64,1488,248.000000,30.906250,23.250000,7.975806
3,Kings XI Punjab,13189,404,10047,1674.500000,32.646040,24.868812,7.876381
4,Delhi Capitals,4415,152,3411,568.500000,29.046053,22.440789,7.766051
5,Royal Challengers Bangalore,15254,515,11878,1979.666667,29.619417,23.064078,7.705338
6,Delhi Daredevils,10709,344,8349,1391.500000,31.130814,24.270349,7.696011
7,Rajasthan Royals,13025,404,10157,1692.833333,32.240099,25.141089,7.694201
8,Mumbai Indians,15954,539,12466,2077.666667,29.599258,23.128015,7.678806
9,Sunrisers Hyderabad,10293,361,8067,1344.500000,28.512465,22.346260,7.655634


In [79]:
dfPowerplayBowling = dfPowerplayOvers.groupby('BowlingTeam').agg({'total_run':'sum','isWicketDelivery':'sum'}).reset_index()

dfPowerplayBowling.columns = ['Team','Runs Conceded','Wickets']
dfPOBallsBowled = dfDeliveries[~((dfDeliveries['extra_type']=='wides'))&(dfDeliveries['PhaseOfPlay']=='PowerPlay Overs')]\
                  .groupby('BowlingTeam')['ID']\
                  .count().reset_index()
dfPOBallsBowled.columns = ['Team','Balls Bowled']

dfPowerplayBowling = dfPowerplayBowling.merge(dfPOBallsBowled, on='Team')


dfPowerplayBowling['Overs'] = dfPowerplayBowling['Balls Bowled']/6

dfPowerplayBowling['Bowling Average'] = dfPowerplayBowling['Runs Conceded']/dfPowerplayBowling['Wickets']
dfPowerplayBowling['Bowling Strike Rate'] = dfPowerplayBowling['Balls Bowled']/dfPowerplayBowling['Wickets']
dfPowerplayBowling['Economy Rate'] = dfPowerplayBowling['Runs Conceded']/dfPowerplayBowling['Overs']
dfPowerplayBowling = dfPowerplayBowling.sort_values('Economy Rate',ascending=False).reset_index(drop=True)

dfPowerplayBowling

,Team,Runs Conceded,Wickets,Balls Bowled,Overs,Bowling Average,Bowling Strike Rate,Economy Rate
0,Gujarat Lions,1501,49,1085,180.833333,30.632653,22.142857,8.300461
1,Lucknow Super Giants,735,23,541,90.166667,31.956522,23.521739,8.151571
2,Rising Pune Supergiants,676,18,500,83.333333,37.555556,27.777778,8.112000
3,Kings XI Punjab,9137,263,6888,1148.000000,34.741445,26.190114,7.959059
4,Rising Pune Supergiant,758,28,576,96.000000,27.071429,20.571429,7.895833
5,Royal Challengers Bangalore,10581,305,8137,1356.166667,34.691803,26.678689,7.802138
6,Delhi Daredevils,7486,222,5757,959.500000,33.720721,25.932432,7.801980
7,Chennai Super Kings,9672,320,7519,1253.166667,30.225000,23.496875,7.718048
8,Gujarat Titans,738,27,577,96.166667,27.333333,21.370370,7.674177
9,Kolkata Knight Riders,10312,288,8078,1346.333333,35.805556,28.048611,7.659322


In [113]:
df = dfDeliveries.iloc[:, :]


In [117]:
import pandas as pd

# Assume df is your DataFrame with the specified columns

# Step 1: Select relevant columns
selected_columns = ['batter', 'bowler', 'batsman_run', 'isWicketDelivery']

# Step 2: Filter data for only relevant rows (e.g., excluding extras)
filtered_data = df[df['extra_type'].isnull()]

# Step 3: Group by 'batter' and 'bowler'
grouped_data = filtered_data[selected_columns].groupby(['batter', 'bowler'])

# Step 4: Aggregate to calculate stats
batsmen_vs_bowler_stats = grouped_data.agg({
    'batsman_run': 'sum',            # Runs scored
    'isWicketDelivery': 'sum'        # Wickets taken
}).reset_index()

# Rename columns for clarity
batsmen_vs_bowler_stats.columns = ['batter', 'bowler', 'runs_scored', 'wickets_taken']

# Add a new column for balls faced
batsmen_vs_bowler_stats['balls_faced'] = grouped_data.size().reset_index(name='balls_faced')['balls_faced']

csv_file_path = r'C:\Users\hp\Desktop\MAJOR\my_code\playervplayer.csv'

# Save the DataFrame to a CSV file
batsmen_vs_bowler_stats.to_csv(csv_file_path, index=False)


In [3]:
import pandas as pd
dfDeliveries = pd.read_csv('./data/IPL_Ball_by_Ball_2008_2022.csv')

In [5]:

import pandas as pd
df = dfDeliveries.iloc[:,:]
df['delivery'] = 1
# Assuming df is your DataFrame with the specified columns
selected_columns = ['batter', 'bowler', 'batsman_run', 'isWicketDelivery']
filtered_data = df[df['extra_type'].isnull()].copy()  # Use copy to avoid SettingWithCopyWarning

# Create new columns for Dot Balls, Singles, Fours, and Sixes
filtered_data['Dot_Balls'] = (filtered_data['batsman_run'] == 0).astype(int)
filtered_data['Singles'] = (filtered_data['batsman_run'] == 1).astype(int)
filtered_data['Fours'] = (filtered_data['batsman_run'] == 4).astype(int)
filtered_data['Sixes'] = (filtered_data['batsman_run'] == 6).astype(int)

# Group by 'batter' and 'bowler'
grouped_data = filtered_data.groupby(['batter', 'bowler'])

# Aggregate to calculate stats
batsmen_vs_bowler_stats = grouped_data.agg({
    'batsman_run': 'sum',    # Runs scored
    'isWicketDelivery': 'sum',   # Wickets taken
    'Dot_Balls': 'sum',     # Dot Balls
    'Singles': 'sum',       # Singles
    'Fours': 'sum',         # Fours
    'Sixes': 'sum'          # Sixes
}).reset_index()

# Rename columns for clarity
batsmen_vs_bowler_stats.columns = ['batter', 'bowler', 'runs_scored', 'wickets_taken', 'dot_balls', 'singles', 'fours', 'sixes']



# Add a new column for balls faced
batsmen_vs_bowler_stats['balls_faced'] = grouped_data.size().reset_index(name='balls_faced')['balls_faced']


batsmen_vs_bowler_stats

,batter,bowler,runs_scored,wickets_taken,dot_balls,singles,fours,sixes,balls_faced
0,A Ashish Reddy,A Nehra,7,1,4,3,1,0,8
1,A Ashish Reddy,AB Dinda,9,0,1,5,1,0,7
2,A Ashish Reddy,AD Mathews,25,0,2,5,1,2,12
3,A Ashish Reddy,AD Russell,4,1,0,2,0,0,3
4,A Ashish Reddy,Anureet Singh,2,0,0,2,0,0,2
...,...,...,...,...,...,...,...,...,...
24119,Z Khan,SR Watson,2,0,2,2,0,0,4
24120,Z Khan,Shakib Al Hasan,3,0,0,3,0,0,3
24121,Z Khan,Sohail Tanvir,3,0,3,1,0,0,5
24122,Z Khan,VRV Singh,0,0,2,0,0,0,2


# To Merge

In [6]:
matches_played_together = pd.read_csv('./derived/matches_played_together.csv')

In [7]:
merged_df = pd.merge(batsmen_vs_bowler_stats,matches_played_together,left_on=['batter', 'bowler'], right_on=['Player1', 'Player2'], how='inner')

In [8]:
merged_df.head(10)

,batter,bowler,runs_scored,wickets_taken,dot_balls,singles,fours,sixes,balls_faced,Player1,Player2,Match IDs,Innings Appearance,Innings Played
0,A Ashish Reddy,A Nehra,7,1,4,3,1,0,8,A Ashish Reddy,A Nehra,"[829773, 598044]","['Both Innings', 'Both Innings']",4
1,A Ashish Reddy,AB Dinda,9,0,1,5,1,0,7,A Ashish Reddy,AB Dinda,"[598018, 598000]","['Both Innings', 'Both Innings']",4
2,A Ashish Reddy,AD Mathews,25,0,2,5,1,2,12,A Ashish Reddy,AD Mathews,"[829731, 598018]","['Both Innings', 'Both Innings']",4
3,A Ashish Reddy,AD Russell,4,1,0,2,0,0,3,A Ashish Reddy,AD Russell,[980915],['Both Innings'],2
4,A Ashish Reddy,Anureet Singh,2,0,0,2,0,0,2,A Ashish Reddy,Anureet Singh,[829759],['Both Innings'],2
5,A Ashish Reddy,Azhar Mahmood,2,0,1,2,0,0,3,A Ashish Reddy,Azhar Mahmood,[598021],['Both Innings'],2
6,A Ashish Reddy,B Kumar,1,0,0,1,0,0,1,A Ashish Reddy,B Kumar,[598000],['Both Innings'],2
7,A Ashish Reddy,BW Hilfenhaus,2,1,1,0,0,0,2,A Ashish Reddy,BW Hilfenhaus,[548352],['Both Innings'],2
8,A Ashish Reddy,CH Morris,11,0,1,2,0,1,5,A Ashish Reddy,CH Morris,"[829727, 598030]","['Both Innings', 'Both Innings']",4
9,A Ashish Reddy,DJ Bravo,24,1,4,4,3,1,13,A Ashish Reddy,DJ Bravo,"[829773, 598030, 548352]","['Both Innings', 'Both Innings', 'Both Innings']",6


In [9]:
merged_df.drop('Innings Appearance', axis=1, inplace=True)


In [10]:
merged_df.head(10)

,batter,bowler,runs_scored,wickets_taken,dot_balls,singles,fours,sixes,balls_faced,Player1,Player2,Match IDs,Innings Played
0,A Ashish Reddy,A Nehra,7,1,4,3,1,0,8,A Ashish Reddy,A Nehra,"[829773, 598044]",4
1,A Ashish Reddy,AB Dinda,9,0,1,5,1,0,7,A Ashish Reddy,AB Dinda,"[598018, 598000]",4
2,A Ashish Reddy,AD Mathews,25,0,2,5,1,2,12,A Ashish Reddy,AD Mathews,"[829731, 598018]",4
3,A Ashish Reddy,AD Russell,4,1,0,2,0,0,3,A Ashish Reddy,AD Russell,[980915],2
4,A Ashish Reddy,Anureet Singh,2,0,0,2,0,0,2,A Ashish Reddy,Anureet Singh,[829759],2
5,A Ashish Reddy,Azhar Mahmood,2,0,1,2,0,0,3,A Ashish Reddy,Azhar Mahmood,[598021],2
6,A Ashish Reddy,B Kumar,1,0,0,1,0,0,1,A Ashish Reddy,B Kumar,[598000],2
7,A Ashish Reddy,BW Hilfenhaus,2,1,1,0,0,0,2,A Ashish Reddy,BW Hilfenhaus,[548352],2
8,A Ashish Reddy,CH Morris,11,0,1,2,0,1,5,A Ashish Reddy,CH Morris,"[829727, 598030]",4
9,A Ashish Reddy,DJ Bravo,24,1,4,4,3,1,13,A Ashish Reddy,DJ Bravo,"[829773, 598030, 548352]",6


In [11]:
merged_df.to_csv('./derived/player_v_player.csv', index=False)

# Done

In [122]:
import pandas as pd

# Assume df is your DataFrame with the specified columns

# Select relevant columns for batting statistics
batting_columns = ['batter', 'batsman_run', 'extras_run', 'Venue']

# Filter data for only relevant rows (e.g., excluding extras)
filtered_data = df[df['extra_type'].isnull()]

# Group by 'Venue' and 'batter'
grouped_data = filtered_data[batting_columns].groupby(['Venue', 'batter'])

# Aggregate to calculate batting stats
batting_stats_by_venue = grouped_data.agg({
    'batsman_run': 'sum',
    'extras_run': 'sum',
}).reset_index()

# Count the number of innings
batting_stats_by_venue['Innings'] = grouped_data.size().reset_index(name='Innings')['Innings']

# Calculate additional statistics like Strike Rate and Batting Average
batting_stats_by_venue['Balls Faced'] = batting_stats_by_venue['batsman_run'] + batting_stats_by_venue['extras_run']
batting_stats_by_venue['Runs'] = batting_stats_by_venue['batsman_run'] + batting_stats_by_venue['extras_run']
batting_stats_by_venue['Dismissals'] = batting_stats_by_venue['batsman_run'].apply(lambda x: 1 if x > 0 else 0)
batting_stats_by_venue['Strike Rate'] = (batting_stats_by_venue['Runs'] / batting_stats_by_venue['Balls Faced']) * 100
batting_stats_by_venue['Batting Average'] = batting_stats_by_venue['Runs'] / batting_stats_by_venue['Dismissals']

# Display the result
batting_stats_by_venue





,Venue,batter,batsman_run,extras_run,Innings,Balls Faced,Runs,Dismissals,Strike Rate,Batting Average
0,Arun Jaitley Stadium,A Mishra,7,0,9,7,7,1,100.0,7.0
1,Arun Jaitley Stadium,AB de Villiers,89,0,56,89,89,1,100.0,89.0
2,Arun Jaitley Stadium,AD Hales,14,0,10,14,14,1,100.0,14.0
3,Arun Jaitley Stadium,AD Russell,110,0,59,110,110,1,100.0,110.0
4,Arun Jaitley Stadium,AJ Finch,2,0,3,2,2,1,100.0,2.0
...,...,...,...,...,...,...,...,...,...,...
5475,"Zayed Cricket Stadium, Abu Dhabi",V Kohli,10,0,8,10,10,1,100.0,10.0
5476,"Zayed Cricket Stadium, Abu Dhabi",VR Iyer,112,0,72,112,112,1,100.0,112.0
5477,"Zayed Cricket Stadium, Abu Dhabi",WP Saha,12,0,13,12,12,1,100.0,12.0
5478,"Zayed Cricket Stadium, Abu Dhabi",YBK Jaiswal,55,0,25,55,55,1,100.0,55.0


In [139]:

csv_file_path = r'C:\Users\hp\Desktop\MAJOR\my_code\full.csv'

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

In [138]:
import pandas as pd

# Assume df is your DataFrame with the specified columns

# Select relevant columns for bowling statistics
bowling_columns = ['bowler', 'batsman_run', 'extras_run', 'isWicketDelivery', 'overs', 'ballnumber', 'non_boundary']

# Filter data for only relevant rows (e.g., excluding extras)
filtered_data = df[df['extra_type'].isnull()]

# Group by 'bowler'
grouped_data = filtered_data[bowling_columns].groupby(['bowler'])
dot_balls = df[df['batsman_run'] == 0 ]
dot_balls_count = dot_balls.groupby(['bowler']).size().reset_index(name='Dot_Balls_Count')

# Display the result
print(dot_balls_count)
# Aggregate to calculate bowling stats
bowling_stats = grouped_data.agg({
    'batsman_run': 'sum',                  # Runs conceded
    'isWicketDelivery': 'sum',              # Wickets
    'overs': 'sum',                         # Overs bowled
    'ballnumber': 'count',                  # Balls bowled
    'non_boundary': 'sum',                  # Boundaries conceded (4s and 6s)
    'overs': 'max'                          # Maximum overs bowled by the bowler in a match
}).reset_index()

# Additional statistics
bowling_stats['Balls Bowled'] = (bowling_stats['overs'] * 6) + bowling_stats['ballnumber']
bowling_stats['Dot Balls'] = bowling_stats['dot_balls_count']
#bowling_stats['4s & 6s Conceded'] = bowling_stats['non_boundary']
bowling_stats['Strike Rate'] = (bowling_stats['Balls Bowled'] / bowling_stats['isWicketDelivery']).replace([np.inf, -np.inf], np.nan).fillna(0)
bowling_stats['Bowling Average'] = (bowling_stats['batsman_run'] / bowling_stats['isWicketDelivery']).replace([np.inf, -np.inf], np.nan).fillna(0)
bowling_stats['Bowling Economy'] = (bowling_stats['batsman_run'] / bowling_stats['overs']).replace([np.inf, -np.inf], np.nan).fillna(0)

# Display the result
bowling_stats





             bowler  Dot_Balls_Count
0    A Ashish Reddy               89
1          A Badoni                4
2        A Chandila              105
3       A Choudhary               49
4       A Dananjaya                7
..              ...              ...
458       YK Pathan              456
459       YS Chahal             1129
460      Yash Dayal               87
461    Yuvraj Singh              282
462          Z Khan             1005

[463 rows x 2 columns]


KeyError: 'dot_balls_count'